# How to Access Pipeline Results - Deep Dive

Each step in the pipeline writes its results in the `Result` object of the pipeline instance (Vanillix, Varix, etc.).  
In this tutorial, we explore how to access and interpret the results.  

The attributes of the `Result` object are mostly instances of a `TrainingDynamics` class.  
This class provides a standardized interface for accessing results from different splits and epochs.  

**IMPORTANT**  
> Epoch-specific `TrainingDynamics`—such as losses or intermediate latent spaces—are not stored every epoch by default.  
> You need to set the `checkpoint_interval` parameter in the config according to your needs.

## What You Will Learn

We go in depth into:

- The `TrainingDynamics` API  
  - latent spaces  
  - losses  
  - reconstructions  
  - sample_ids  

- Nested `TrainingDynamics` like `sub_losses`  

- Non-`TrainingDynamics` result attributes, such as:  
  - datasets  
  - new_datasets  
  - model  
  - adata_latent  
  - final_reconstruction  
  - embedding_evaluation  

- Special methods to obtain pandas DataFrames  
  - get latent space as a DataFrame with `sample_ids`  
  - get reconstruction as a DataFrame with `sample_ids`  

## 1) Filling the Result Object

Before we can investigate the result object, we first need to create results.  
Therefore, we run two pipelines: `XModalix` and `Varix`.

### 1.1 The Datasets

For the `Varix` example, we use a mock single-cell dataset as a `MuData` object inside our custom `DataPackage` class.  
For our `XModalix` example, we use the same dataset as in the `XModalix.ipynb` tutorial.  

As a showcase for data modality translation with `XModalix`, we use cancer gene expression from TCGA in combination with handwritten digits from the [MNIST dataset](https://keras.io/api/datasets/mnist/).  
Our goal is to translate the gene expression signature of five selected cancer subtypes to images of digits, where each cancer subtype class is assigned a digit between 0-4.  

In practice, these images could be histopathological images or any other data modality.  
Before showing data preparation and `XModalix` training, here is some background on the basic idea of a cross-modal VAE as proposed by [Yang & Uhler](https://arxiv.org/abs/1902.03515).

### ❗❗ Requirements: Getting Tutorial Data ❗❗

You can use the following bash commands to download the data and set up the correct folder structure.  
**Assumption:** you are in the root of `autoencodix_package`.

```bash
mkdir -p data
cd data
wget "https://cloud.scadsai.uni-leipzig.de/index.php/s/bq64MaQyZGZfN64/download/XModalix-Tut-data.zip"
unzip XModalix-Tut-data.zip
```



In [1]:
import os

p = os.getcwd()
d = "autoencodix_package"
if d not in p:
    raise FileNotFoundError(f"'{d}' not found in path: {p}")
os.chdir(os.sep.join(p.split(os.sep)[: p.split(os.sep).index(d) + 1]))
print(f"Changed to: {os.getcwd()}")


Changed to: /Users/maximilianjoas/development/autoencodix_package


In [2]:
%%capture
from autoencodix.utils.example_data import EXAMPLE_MULTI_SC
from autoencodix.configs.varix_config import VarixConfig
from autoencodix.configs.default_config import DataCase, DataInfo, DataConfig
from autoencodix.configs.xmodalix_config import XModalixConfig
import autoencodix as acx

varix_config = VarixConfig(
    learning_rate=0.001,
    epochs=33,
    checkpoint_interval=1,
    default_vae_loss="kl",  # kl or mmd possible
    data_case=DataCase.MULTI_SINGLE_CELL,
)
varix = acx.Varix(data=EXAMPLE_MULTI_SC, config=varix_config)
result = varix.run()

# XModalix
clin_file = os.path.join("data/XModalix-Tut-data/combined_clin_formatted.parquet")
rna_file = os.path.join("data/XModalix-Tut-data/combined_rnaseq_formatted.parquet")
img_root = os.path.join("data/XModalix-Tut-data/images/tcga_fake")

xmodalix_config = XModalixConfig(
    checkpoint_interval=1,
    class_param="CANCER_TYPE_ACRONYM",
    epochs=30,
    data_case=DataCase.IMG_TO_BULK,
    data_config=DataConfig(
        data_info={
            "img": DataInfo(
                file_path=img_root,
                data_type="IMG",
                scaling="MINMAX",
                translate_direction="to",
            ),
            "rna": DataInfo(
                file_path=rna_file,
                data_type="NUMERIC",
                scaling="MINMAX",
                translate_direction="from",
            ),
            "anno": DataInfo(file_path=clin_file, data_type="ANNOTATION", sep="\t"),
        },
    ),
)

xmodalix = acx.XModalix(config=xmodalix_config)
xmodalix_result = xmodalix.run()

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## 2) TrainingDynamics Interface Deep Dive
Before accessing the actual results, we provide a theory section on our interface:  
<br><br>
The `TrainingDynamics` object has the following form:  
`<epoch><split><data>`  

So, if you want to access the train loss for the 5th epoch, you would use:  
```python
result.loss.get(epoch=5, split="train")
````

##### The `.get()` Method Explained
Let's say, we're interessted in thre reconstructions of our autoencoder.  
The `reconstructions.get()` method provides flexible access to reconstruction data stored during training. It can retrieve data for specific epochs, specific splits, or any combination of these parameters.

##### Parameters

* **`epoch`** (Optional[int]):

  * Positive integer (e.g., `2`): Get reconstructions from that specific epoch
  * Negative integer (e.g., `-1`): Get the latest epoch (-1), second-to-last (-2), etc.
  * `None`: Return data for all epochs

* **`split`** (Optional[str]):

  * Valid values: `"train"`, `"valid"`, `"test"`
  * `None`: Return data for all splits

##### Return Value Behavior

The method returns different types depending on the parameters:

1. **Both `epoch` and `split` specified**:

   * Returns a NumPy array for that specific epoch and split
   * Example: `get(epoch=2, split="train")` → `array([...])`

2. **Only `epoch` specified**:

   * Returns a dictionary of all splits for that epoch
   * Example: `get(epoch=2)` → `{"train": array([...]), "valid": array([...]), ...}`

3. **Only `split` specified**:

   * Returns a NumPy array containing data for that split across all epochs
   * Example: `get(split="train")` → `array([[...], [...], ...])` (first dimension represents epochs)

4. **Neither specified**:

   * Returns the complete nested dictionary structure
   * Example: `get()` → `{0: {"train": array([...])}, 1: {...}, ...}`

##### Special Handling

* If an invalid split is provided, a `KeyError` is raised
* Negative epoch indices work like Python list indexing (-1 is the last epoch)
* If an epoch doesn't exist, an empty array or dictionary is returned

##### Code Example

```python
# Access train reconstructions for the 5th epoch
train_epoch_5 = result.reconstructions.get(epoch=5, split="train")

# Access all splits for the latest epoch
latest_epoch_all_splits = result.reconstructions.get(epoch=-1)

# Access data for all epochs for the "valid" split
all_epochs_valid = result.reconstructions.get(split="valid")

# Access the full nested dictionary
full_data = result.reconstructions.get()
```


## 3) Working with Actual Results
### 3.1) Varix

Exemplary, we show how to get the `latentspaces` of the `result` attribute and to access different loss types.

In [3]:
all_ls = result.latentspaces.get()
print(f"Keys of all latentspaces: {all_ls.keys()}")


Keys of all latentspaces: dict_keys([-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])


We see that we have latent spaces for each epoch because we set `checkpoint_interval=1` in our configs in [Step 1](#1-filling-the-result-object).  

For each epoch, we have the latent space for the `train` and `valid` splits. The `-1` epoch is a special key for the `test` split. For the other splits, negative indexing works as in Python lists: `-1` gives the last epoch, `-2` the second-to-last, and so on.  

**Special Case**  
> You cannot get the last epoch for all splits at once. You can either get the last epoch for `train` and `valid`, or only for `test`. See code below.


In [4]:
print(f"Splits in 2. epoch: {result.latentspaces.get(epoch=2).keys()}")

# this will only give you the data for train and valid, since 'test' is a special case
print(f"Splits in epoch =-1: {result.latentspaces.get(epoch=-1).keys()}")
# Get test by adding the 'split' argument.
test_ls = result.latentspaces.get(split="test")
print(test_ls[0][0])
# get a specific epoch and specific split:

print("\n")
print("-"*80)

print(f"latentspace of one sample in train split at epoch 4: {result.latentspaces.get(split="train", epoch=4)[0]}")

Splits in 2. epoch: dict_keys(['train', 'valid'])
Splits in epoch =-1: dict_keys(['train', 'valid'])
[ 2.6214824   4.6263213   1.1570647  -0.05911068 -1.5539709   3.948379
  3.979298   -0.49932882  0.37956843  0.19487692 -2.219246    1.6172006
 -0.41644672 -0.75258625 -1.9791765   0.42172816]


--------------------------------------------------------------------------------
latentspace of one sample in train split at epoch 4: [ 2.2433555  -1.4524041  -1.6737837  -0.1882257  -0.78592366  0.8265473
  0.23994362 -1.0492905  -1.6316438   1.4425147  -0.14815688  1.3808663
 -0.7087234  -0.2454678  -0.369424    0.7728152 ]


### 3.2 XModalix

Accessing the results works slightly differently for `XModalix`, because we have:

1. **Multiple latent spaces** (one for each data modality).  

2. **Translations and reference translations**, not just reconstructions.  
   - These are reconstructions within one data modality on the same split as the translation. For example, if we translate from `rna` to `img`, the reference translation would be the reconstruction from `img` to `img`.  
   - You can access these `translations` via the `reconstruction` attribute of the `result` object. First, apply the usual `TrainingDynamics` API via `get()`, then you get a dictionary for each data modality with translations and reference translations.

3. **Multiple losses**.  
   - These are accessed via the `sub_losses` attribute of the `result` object, which is a dictionary of `TrainingDynamics`. First, select the loss type you're interested in, then work with the usual `TrainingDynamics` API.  

   **Note on naming sub-losses:**  
   - Global losses are simply named after the loss type, e.g., `class_loss`.  
   - Losses per data modality are named with the following convention:  
     ```
     <global_data_modality_key>.<specific_data_modality_name>.<loss_name>
     ```  
     For example: `multi_bulk.rna.class_loss` or `multi_sc.celltype.reconstruction_loss`. See print below.


#### 3.2.1 Access Modality Latent Spaces

As described in our `XModalix Deep Dive` [1], we fit one latent space per data modality.  
You can access this by first selecting the epoch and split you're interested in (standard `TrainingDynamics` API).  
The result will be a `Dict` with the name of each data modality as the key.


In [5]:
print(f" Keys of data modalities for latent space dynamic: {xmodalix_result.latentspaces.get(epoch=-1, split='test').keys()}")


 Keys of data modalities for latent space dynamic: dict_keys(['multi_bulk.rna', 'img.img'])


Now you can access the latent space of the `image` modality with the key `img.img`

In [6]:
xmodalix_result.latentspaces.get(epoch=-1, split="test").get("img.img")

array([[-0.03141681,  0.16931348,  0.3611572 , ..., -1.9819316 ,
         6.0393643 , -1.4682697 ],
       [-0.5428814 ,  0.14300315, -1.2133503 , ...,  0.95548046,
         2.1448107 , -0.8962388 ],
       [-1.3134758 , -1.6307245 , -0.12368247, ...,  1.6917285 ,
         4.036174  ,  1.5661058 ],
       ...,
       [-0.5011886 , -0.99498355, -0.9280783 , ...,  4.3056226 ,
        -0.52255327, -1.0871631 ],
       [-1.2227187 ,  0.41767797,  0.36134344, ...,  0.803061  ,
         0.6684158 , -2.9062355 ],
       [-0.43811584, -0.25479415, -0.34566346, ..., -1.3903735 ,
         2.8972611 ,  1.1275443 ]], shape=(646, 16), dtype=float32)

#### 3.2.2 Access Translation

In [7]:
print("Get reconstruction keys")
# Frist define split and epoch you're interested in
# usually test split (there are no epochs, so by default this is always epoch=-1)
recons = xmodalix_result.reconstructions.get(split="test", epoch=-1)
print(recons.keys())
print("Getting Translation")
trans = recons.get("translation")
print(f"shape of translation: {trans.shape}")

Get reconstruction keys
dict_keys(['multi_bulk.rna', 'img.img', 'translation', 'reference_img.img_to_img.img'])
Getting Translation
shape of translation: (711, 1, 64, 64)


#### 3.2.3 Access Sub-Losses

In [8]:

sub_losses = xmodalix_result.sub_losses
print("Sub Losses:")
print(f"keys: {sub_losses.keys()}")
print("\n")
recon_dyn = sub_losses.get(key="paired_loss")
print("Value of paired loss in epoch 4 for train split")
print(f"{recon_dyn.get(split='train', epoch=4):.2f}")


Sub Losses:
keys: dict_keys(['adver_loss', 'aggregated_sub_losses', 'paired_loss', 'class_loss', 'multi_bulk.rna.recon_loss', 'multi_bulk.rna.var_loss', 'multi_bulk.rna.anneal_factor', 'multi_bulk.rna.effective_beta_factor', 'multi_bulk.rna.loss', 'img.img.recon_loss', 'img.img.var_loss', 'img.img.anneal_factor', 'img.img.effective_beta_factor', 'img.img.loss', 'clf_loss'])


Value of paired loss in epoch 4 for train split
6.91


## 4) Non-TrainingDynamics Result Attributes

There are other (intermediate) results that are not created during training but might still be interesting.  
These results do not follow a uniform interface like `TrainingDynamics`, but are often more straightforward. We go over each attribute quickly.

#### 4.1 Datasets

The `datasets` attribute stores the preprocessed data in a `DatasetContainer`.  
This is basically a dict with `train`, `valid`, and `test` as keys, and each value is a child class of a PyTorch dataset.  
Whenever you need to re-access your preprocessed data, you can do so using the `datasets` attribute, as shown below:


In [9]:
print(result.datasets)
print(result.datasets.train)
print(type(result.datasets.train.data))
result.datasets.train.metadata.head()

DatasetContainer(train=<autoencodix.data._numeric_dataset.NumericDataset object at 0x142d73bc0>, valid=<autoencodix.data._numeric_dataset.NumericDataset object at 0x142d73f20>, test=<autoencodix.data._numeric_dataset.NumericDataset object at 0x142d73a70>)
<class 'torch.Tensor'>


,rna:cell_type,rna:batch,rna:donor,rna:cell_cycle,rna:n_genes,protein:cell_type,protein:batch,protein:donor,protein:cell_cycle,protein:n_genes
cell_1,type_0,batch3,donor4,G1,357,type_0,batch3,donor4,G1,147
cell_107,type_4,batch2,donor3,S,335,type_4,batch2,donor3,S,146
cell_189,type_3,batch2,donor2,G2M,368,type_3,batch2,donor2,G2M,155
cell_19,type_3,batch1,donor1,G1,345,type_3,batch1,donor1,G1,147
cell_191,type_0,batch3,donor2,S,349,type_0,batch3,donor2,S,150


#### 4.2 New Datasets

Whenever you run the `predict` step of the pipeline and pass new, unseen data to it, we preprocess this data (if necessary).  
To avoid overwriting the original `datasets`, we store this in `new_datasets`.  

If you run `predict` again with other data, `new_datasets` will be overridden.  
Otherwise, `new_datasets` works the same way as `datasets`.


First we create new data and then we run predict.

In [10]:
import copy

from autoencodix.utils.example_data import EXAMPLE_MULTI_SC
new_data = copy.copy(EXAMPLE_MULTI_SC)
new_multi_sc = new_data.multi_sc["multi_sc"]
for modname, mod in new_multi_sc.mod.items():
    new_names = mod.obs_names.str.replace('cell', 'new_cell')
    mod.index = new_names

    mod.obs_names = new_names
    print(mod.index)
    new_multi_sc.mod[modname] = mod

new_data.multi_sc["multi_sc"] = new_multi_sc
new_data.multi_sc["multi_sc"].update()

Index(['new_cell_0', 'new_cell_1', 'new_cell_10', 'new_cell_100',
       'new_cell_101', 'new_cell_102', 'new_cell_103', 'new_cell_104',
       'new_cell_105', 'new_cell_106',
       ...
       'new_cell_990', 'new_cell_991', 'new_cell_992', 'new_cell_993',
       'new_cell_994', 'new_cell_995', 'new_cell_996', 'new_cell_997',
       'new_cell_998', 'new_cell_999'],
      dtype='object', length=1000)
Index(['new_cell_0', 'new_cell_1', 'new_cell_10', 'new_cell_100',
       'new_cell_101', 'new_cell_102', 'new_cell_103', 'new_cell_104',
       'new_cell_105', 'new_cell_106',
       ...
       'new_cell_990', 'new_cell_991', 'new_cell_992', 'new_cell_993',
       'new_cell_994', 'new_cell_995', 'new_cell_996', 'new_cell_997',
       'new_cell_998', 'new_cell_999'],
      dtype='object', length=1000)


/Users/maximilianjoas/development/autoencodix_package/.venv/lib/python3.12/site-packages/mudata/_core/mudata.py:758: ImplicitModificationWarning: Setting element `.varm['rna']` of view, initializing view as actual.
  attrm[mod] = mapping > 0
/Users/maximilianjoas/development/autoencodix_package/.venv/lib/python3.12/site-packages/mudata/_core/mudata.py:758: ImplicitModificationWarning: Setting element `.varm['protein']` of view, initializing view as actual.
  attrm[mod] = mapping > 0


Run the predict step:

In [11]:
%%capture
varix.predict(data=new_data)

Examine `datasets` and `new_datasets`:  
We see that `datasets` still is kept and only new_datasets is updated.

In [12]:
print(f"Sample of original dataset: {result.datasets.train.sample_ids[0]}")

print(f"Sample of new dataset: {result.new_datasets.test.sample_ids[0]}")

Sample of original dataset: cell_1
Sample of new dataset: new_cell_0


#### 4.3 Model Attribute
This is straightforward the trained model as PytTorch Module.

In [13]:
result.model

VarixArchitecture(
  (_encoder): Sequential(
    (0): Linear(in_features=20, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=16, bias=True)
    (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.1, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=16, out_features=16, bias=True)
    (9): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Dropout(p=0.1, inplace=False)
    (11): ReLU()
  )
  (_mu): Linear(in_features=16, out_features=16, bias=True)
  (_logvar): Linear(in_features=16, out_features=16, bias=True)
  (_decoder): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.1, inplace=False)

#### 4.4 Adata Latent

We save the latent space of the `test` split from the final trained model as an `AnnData` object for the single-cell community.  

This is also useful for non-single-cell cases, because you can still obtain the sample IDs via `.obs`.


In [14]:
print(result.adata_latent)
print(result.adata_latent.obs)

AnnData object with n_obs × n_vars = 1000 × 16
    uns: 'var_names'
Empty DataFrame
Columns: []
Index: [new_cell_0, new_cell_1, new_cell_10, new_cell_100, new_cell_101, new_cell_102, new_cell_103, new_cell_104, new_cell_105, new_cell_106, new_cell_107, new_cell_108, new_cell_109, new_cell_11, new_cell_110, new_cell_111, new_cell_112, new_cell_113, new_cell_114, new_cell_115, new_cell_116, new_cell_117, new_cell_118, new_cell_119, new_cell_12, new_cell_120, new_cell_121, new_cell_122, new_cell_123, new_cell_124, new_cell_125, new_cell_126, new_cell_127, new_cell_128, new_cell_129, new_cell_13, new_cell_130, new_cell_131, new_cell_132, new_cell_133, new_cell_134, new_cell_135, new_cell_136, new_cell_137, new_cell_138, new_cell_139, new_cell_14, new_cell_140, new_cell_141, new_cell_142, new_cell_143, new_cell_144, new_cell_145, new_cell_146, new_cell_147, new_cell_148, new_cell_149, new_cell_15, new_cell_150, new_cell_151, new_cell_152, new_cell_153, new_cell_154, new_cell_155, new_cell_1

#### 4.5 Final Reconstruction
This attribute gives you the exact data structure as you used for input i.e. `MuData` in our case, with the reconstructed values.

In [15]:
result.final_reconstruction

MuData object with n_obs × n_vars = 1000 × 20
  2 modalities
    rna:	1000 x 10
      obs:	'rna:cell_type', 'rna:batch', 'rna:donor', 'rna:cell_cycle', 'rna:n_genes', 'protein:cell_type', 'protein:batch', 'protein:donor', 'protein:cell_cycle', 'protein:n_genes'
    protein:	1000 x 10
      obs:	'rna:cell_type', 'rna:batch', 'rna:donor', 'rna:cell_cycle', 'rna:n_genes', 'protein:cell_type', 'protein:batch', 'protein:donor', 'protein:cell_cycle', 'protein:n_genes'

#### 4.6 Evaluation Embeddings

Before we can access this attribute, we first need to run the `evaluate` step. This will use the latent space and train a downstream machine learning task. In our case, we want to classify the cancer type.  

The results of this evaluate step will be stored in `embedding_evaluation`.


In [16]:
%%capture
xmodalix.evaluate(params=["CANCER_TYPE"])

In [17]:
xmodalix_result.embedding_evaluation

,score_split,CLINIC_PARAM,metric,value,ML_ALG,ML_TYPE,MODALITY,ML_TASK,ML_SUBTASK
0,train,CANCER_TYPE,roc_auc_ovo,0.906995,LogisticRegression(),classification,multi_bulk.rna,Latent,Latent_$_multi_bulk.rna
1,valid,CANCER_TYPE,roc_auc_ovo,0.933789,LogisticRegression(),classification,multi_bulk.rna,Latent,Latent_$_multi_bulk.rna
2,test,CANCER_TYPE,roc_auc_ovo,0.928152,LogisticRegression(),classification,multi_bulk.rna,Latent,Latent_$_multi_bulk.rna
0,train,CANCER_TYPE,roc_auc_ovo,0.989023,LogisticRegression(),classification,img.img,Latent,Latent_$_img.img
1,valid,CANCER_TYPE,roc_auc_ovo,0.987592,LogisticRegression(),classification,img.img,Latent,Latent_$_img.img
2,test,CANCER_TYPE,roc_auc_ovo,0.983939,LogisticRegression(),classification,img.img,Latent,Latent_$_img.img


In [18]:

varix.result.datasets.test.metadata.head()
varix.evaluate(params=["rna:batch"])

Perform ML task with feature df: Latent
Latent
Perform ML task for target parameter: rna:batch
Perform ML task with feature df: Latent
Latent
Perform ML task for target parameter: rna:batch


Result Object Public Attributes:
------------------------------
latentspaces: TrainingDynamics object
sample_ids: TrainingDynamics object
reconstructions: TrainingDynamics object
mus: TrainingDynamics object
sigmas: TrainingDynamics object
losses: TrainingDynamics object
sub_losses: LossRegistry(_losses={'recon_loss': TrainingDynamics(), 'var_loss': TrainingDynamics(), 'anneal_factor': TrainingDynamics(), 'effective_beta_factor': TrainingDynamics()})
preprocessed_data: Tensor of shape (0,)
model: VarixArchitecture
model_checkpoints: TrainingDynamics object
datasets: DatasetContainer(train=<autoencodix.data._numeric_dataset.NumericDataset object at 0x142d73bc0>, valid=<autoencodix.data._numeric_dataset.NumericDataset object at 0x142d73f20>, test=<autoencodix.data._numeric_dataset.NumericDataset object at 0x141937770>)
new_datasets: DatasetContainer(train=<autoencodix.data._numeric_dataset.NumericDataset object at 0x142d3b350>, valid=None, test=<autoencodix.data._numeric_dataset.NumericD

##  5 Special Methods to Obtain DataFrames
As you've  seen in the [Training Dynamics Section](#2-trainingdynamics-interface-deep-dive), we only get plain values of reconstructions and latetnspaces. Often it is more useful to have sample ids, too. We could obtain the sample ids in the same order via the `sample_ids` TrainingDynamic. To make this more accessible, we added the methods
`get_latent_df` and `get_reconstructions_df`. Here you pass `epoch` and `split` as seen before and you get a pandas DataFrame for the specific split and epoch for the latent space or the reconstruction.

In [19]:
result.get_latent_df(epoch=-1, split="test").head()

,LatDim_0,LatDim_1,LatDim_2,LatDim_3,LatDim_4,LatDim_5,LatDim_6,LatDim_7,LatDim_8,LatDim_9,LatDim_10,LatDim_11,LatDim_12,LatDim_13,LatDim_14,LatDim_15
new_cell_0,3.918209,3.294712,0.276736,-0.022310,0.458934,3.645692,2.111589,1.898614,-1.590129,-0.359401,-1.373160,1.922754,1.125993,-0.843492,-1.129925,-0.234048
new_cell_1,2.456972,4.616240,-0.264957,-0.993260,2.581688,3.342378,2.403553,-0.057253,-0.007646,-0.674661,-0.242371,0.721174,-1.366866,-1.015966,0.282577,0.337071
new_cell_10,0.223041,0.934687,0.101731,-0.348585,-0.330824,0.758900,1.189915,-0.366939,-0.007493,-1.649518,1.446973,1.219632,0.447651,0.177679,0.009620,0.873264
new_cell_100,-1.854113,-1.283103,2.491228,-1.306956,-1.332794,-0.139246,1.137050,0.147499,1.669023,0.631388,0.346797,2.031549,-0.960792,-1.765558,0.246246,-1.518077
new_cell_101,4.460937,1.412730,-0.462453,-0.175874,-1.138595,3.696313,3.163703,0.148417,-1.202437,0.719220,0.277123,2.112525,0.709418,1.093101,1.827790,-0.573955


In [20]:
result.get_reconstructions_df(epoch=-1, split="test").head()

,gene_280,gene_485,gene_115,gene_382,gene_165,gene_479,gene_345,gene_81,gene_436,gene_153,protein_101,protein_146,protein_157,protein_155,protein_183,protein_28,protein_89,protein_116,protein_54,protein_4
new_cell_0,-0.694922,-0.707434,-0.788046,-0.772559,-0.747675,-0.726379,-0.861443,-0.492340,1.355242,1.420983,1.669617,-0.755928,-1.100585,1.514432,1.676864,-0.990130,0.567872,-0.045624,-0.659986,-1.113965
new_cell_1,-0.674102,-0.711368,-0.789275,-0.758618,-0.739204,-0.730103,-0.838547,-0.471383,1.334556,1.419258,1.666623,-0.738986,-1.068399,1.514256,1.669015,-0.974387,0.581663,-0.063435,-0.629224,-1.087513
new_cell_10,0.509440,0.503498,0.349776,0.276923,0.346317,0.396281,0.166480,0.369108,-0.311564,-0.206180,-0.272627,0.540599,0.561846,-0.370599,-0.377840,0.599443,0.408415,-0.362103,0.665120,0.494489
new_cell_100,0.629542,0.624343,0.594572,0.494752,0.681353,0.666638,0.438097,0.516011,-0.546351,-0.488707,-0.552897,0.893472,0.907632,-0.749991,-0.536840,0.898068,0.480714,-0.568045,0.839276,0.839763
new_cell_101,-0.839199,-0.655231,-0.772520,-0.785155,-0.735730,-0.681882,-0.897330,-0.583975,1.321200,1.351074,1.521867,-0.787227,-1.183686,1.383675,1.621076,-1.042762,0.409781,0.090534,-0.868935,-1.190879
